<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Sparse_AIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [5]:
pip install pgmpy

In [39]:
import numpy as np
import pandas as pd
from pgmpy.estimators import HillClimbSearch, AICScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from tabulate import tabulate

# Bayesian Network Data Generation 500, ..., 20000 Samples (dense)

In [62]:
# Function to generate sparse CPDs
def generate_sparse_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random sparse probabilities for EI given IR (introduce zeros or very small values)
    ei_given_ir_probs = np.random.rand(3, 3)
    ei_given_ir_probs[ei_given_ir_probs < 0.5] = 0  # Introduce sparsity; any values less than 0.5 are = 0
    ei_given_ir_probs_sum = ei_given_ir_probs.sum(axis=0, keepdims=True)

    # If any column sums to zero, replace it with uniform probabilities to avoid NaN
    ei_given_ir_probs[:, ei_given_ir_probs_sum[0] == 0] = 1 / 3
    ei_given_ir_probs /= ei_given_ir_probs.sum(axis=0, keepdims=True)

    # Generate random sparse probabilities for SP given IR and EI
    sp_probs = np.random.rand(3, 3, 3)
    sp_probs[sp_probs < 0.2] = 0  # Introduce sparsity
    sp_probs_sum = sp_probs.sum(axis=0, keepdims=True)

    # If any column sums to zero, replace it with uniform probabilities to avoid NaN
    sp_probs[:, sp_probs_sum[0] == 0] = 1 / 3
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, ei_given_ir_probs, sp_probs

# Function to generate and save samples
def generate_and_save_sparse_samples(ir_probs, ei_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample `IR` state based on `IR` probabilities
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample `EI` state based on `EI` probabilities given `IR`
        ei_probs_given_ir = ei_probs[:, ir_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir[ei_state_idx]

        # Sample `SP` state based on `SP` probabilities given `IR` and `EI`
        sp_probs_given_ir_ei = sp_probs[:, ir_state_idx, ei_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei[sp_state_idx]

        # Append sample data to output list
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}'
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated sparse samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes from 50 to 20000
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

for size in sample_sizes:
    # Generate the sparse CPDs
    ir_probs, ei_given_ir_probs, sp_probs = generate_sparse_cpds()

    # Generate and save individual sparse samples for the given sample size
    generate_and_save_sparse_samples(ir_probs, ei_given_ir_probs, sp_probs, size, f'combined_probabilities_sparse_{size}.csv')

# Notify the user that the process is done
print("\nGeneration and saving of individual sparse samples complete for all sample sizes!")


Sample size: 50 - First few rows of generated sparse samples:

+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|    | IR_State   |   IR_Prob | EI_State   |   EI_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability |
+====+============+===========+============+===========+=================================================+===================+=========================+
|  0 | low        |    0.4488 | good       |         1 | 0.4338, 0.3658, 0.2005                          | stable            |                  0.3658 |
+----+------------+-----------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|  1 | low        |    0.4488 | good       |         1 | 0.4338, 0.3658, 0.2005                          | increase          |                  0.2005 |
+----+------------

# LBN Sparse BIC & Entropy

In [63]:
# Sample sizes to loop through
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

# Initialize list to store K-L divergence and standard deviation values for each sample size
results = []

# Small smoothing value to avoid zero probabilities
epsilon = 1e-10

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dense dataset for the current sample size
    sparse_data_file = f'combined_probabilities_sparse_{sample_size}.csv'
    df_sparse = pd.read_csv(sparse_data_file)

    # Manually encode categorical variables for IR, EI, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df_sparse['IR_encoded'] = df_sparse['IR_State'].map(ir_map)
    df_sparse['EI_encoded'] = df_sparse['EI_State'].map(ei_map)
    df_sparse['SP_encoded'] = df_sparse['Chosen_SP_State'].map(sp_map)

    # Split the data into training, validation, and test sets
    X = df_sparse[['IR_encoded', 'EI_encoded']]
    y = df_sparse['SP_encoded']

    # Split into training (70%) and temp (30%) for validation and test
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

    # Concatenate X and y to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)
    print("Training data:")
    print(df_train.head())

    # Enforce the inclusion of `SP_encoded` using priors (fixed edges)
    priors = [
        ('IR_encoded', 'SP_encoded'),
        ('EI_encoded', 'SP_encoded')
    ]

    # Perform Hill Climb Search with priors (fixed edges)
    hc = HillClimbSearch(df_train)
    best_dag = hc.estimate(scoring_method=AICScore(df_train), fixed_edges=priors)

    # Initialize BayesianNetwork and print edges
    best_model = BayesianNetwork(best_dag.edges())
    print("Learned structure (edges):", best_model.edges())

    if len(best_model.edges()) == 0:
        print("No edges learned. Skipping to next sample size.")
        continue

    # Try fitting parameters using BayesianEstimator
    try:
        best_model.fit(df_train, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=5)
        print("Model fitted successfully.")
    except Exception as e:
        print("Error during fitting:", str(e))
        continue

    # Check model validity
    if not best_model.check_model():
        print("Model check failed.")
        continue

    # --- K-L Divergence and Standard Deviation Calculation Block ---
    # Predict on test data
    try:
        inference = best_model.predict(X_test)
        predicted_probabilities = inference['SP_encoded']

        # Calculate K-L divergence and standard deviation between the ground truth and learned BN
        ground_truth_probabilities = y_test.value_counts(normalize=True).sort_index()
        predicted_probabilities = predicted_probabilities.value_counts(normalize=True).sort_index()

        # Reindex both distributions to have the same set of categories and add smoothing
        all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
        ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
        predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

        # Calculate K-L divergence with smoothing
        kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)

        # Standard deviation between predicted probabilities and actual probabilities
        std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

        # Append results for this sample size
        results.append({
            'Sample_Size': sample_size,
            'K-L_Divergence': kl_divergence,
            'Standard_Deviation': std_dev
        })

        # Print the K-L divergence and standard deviation for this sample size
        print(f"\nResults for sample size {sample_size}:")
        print(f"K-L Divergence: {kl_divergence:.4f}")
        print(f"Standard Deviation: {std_dev:.4f}")

    except Exception as e:
        print("Error during prediction:", str(e))
        continue

# After processing all sample sizes, save results to CSV and display them
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_aic_sparse_results.csv', index=False)

# Print all results after saving to CSV
print("\nAll results have been saved to 'kl_std_aic_sparse_results.csv'.")
print(results_df)


Processing sample size: 50
Training data:
    IR_encoded  EI_encoded  SP_encoded
6            0           2           1
41           1           0           1
46           2           2           0
47           0           2           1
15           1           1           0


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('IR_encoded', 'EI_encoded'), ('EI_encoded', 'SP_encoded')]
Model fitted successfully.


  0%|          | 0/3 [00:00<?, ?it/s]


Results for sample size 50:
K-L Divergence: 2.9370
Standard Deviation: 0.3680

Processing sample size: 100
Training data:
    IR_encoded  EI_encoded  SP_encoded
11           1           2           2
47           0           1           1
85           0           1           0
28           0           0           1
93           2           0           2


...


Results for sample size 19000:
K-L Divergence: 0.0102
Standard Deviation: 0.0492

Processing sample size: 20000
Training data:
       IR_encoded  EI_encoded  SP_encoded
17218           0           1           1
15188           2           2           2
11295           2           2           2
19772           2           2           2
13072           2           2           2


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('EI_encoded', 'SP_encoded'), ('EI_encoded', 'IR_encoded')]
Model fitted successfully.


  0%|          | 0/5 [00:00<?, ?it/s]


Results for sample size 20000:
K-L Divergence: 0.0100
Standard Deviation: 0.0441

All results have been saved to 'kl_std_aic_sparse_results.csv'.
    Sample_Size  K-L_Divergence  Standard_Deviation
0            50        2.936982            0.367990
1           100        0.229440            0.196261
2           150        1.680112            0.374014
3           200        0.136841            0.163299
4           250        0.093774            0.140898
5           300        0.260512            0.191163
6           350        0.179595            0.162308
7           400        0.045546            0.082776
8           450        5.841919            0.201994
9           500        1.578572            0.094907
10          550        0.065821            0.127885
11          600        0.076281            0.089351
12          650        0.407741            0.268557
13          700        0.055888            0.101090
14          750        3.724583            0.237898
15          800      